# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import cartopy.crs as ccrs
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nuuk,64.9876,-52.3907,276.73,90,100,7.64,GL,1719668267
1,1,jamestown,66.7768,151.4367,282.52,80,100,2.82,RU,1719668305
2,2,puerto natales,-16.4042,-6.3494,293.84,71,40,6.50,SH,1719668307
3,3,bredasdorp,50.2266,72.6584,297.55,36,6,2.99,KZ,1719668313
4,4,hithadhoo,60.9053,-92.5890,275.71,84,23,3.42,CA,1719668325


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',  # X-axis (longitude)
    'Lat',  # Y-axis (latitude)
    geo=True,  # Enable geographic plotting
    tiles='OSM',  # Use OpenStreetMap for the world map background
    size='Humidity',  # Size of points based on humidity
    color='blue',  # Color of the points
    hover_cols=['Humidity'],  # Columns for hover information
    frame_width=800,
    frame_height=600,
    title='City Weather Data'
).opts(projection=ccrs.PlateCarree())  # Set the projection to PlateCarree

# Display the map plot 
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df.loc[
    (city_data_df['Humidity'] > 70) &
    (city_data_df['Max Temp'] >= 290) &
    (city_data_df['Max Temp'] <= 300)
]

# Drop any rows with null values
cleaned_criteria_df = criteria_df.dropna()
# Display sample data
cleaned_criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,puerto natales,-16.4042,-6.3494,293.84,71,40,6.50,SH,1719668307
11,11,akureyri,-19.0811,-174.6211,296.93,90,93,5.79,TO,1719668356
28,28,yuzhno-kurilsk,21.5457,-157.0877,297.03,72,0,8.48,US,1719668483
39,39,kataragama,-14.4469,-67.7661,294.85,77,42,1.60,BO,1719668577
53,53,panaba,46.8121,123.3362,292.64,96,100,2.73,CN,1719668698


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nuuk,GL,64.9876,-52.3907,90,
1,jamestown,RU,66.7768,151.4367,80,
2,puerto natales,SH,-16.4042,-6.3494,71,
3,bredasdorp,KZ,50.2266,72.6584,36,
4,hithadhoo,CA,60.9053,-92.5890,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 50000  # Example radius in meters
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,  
    "limit": 1
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nuuk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
bredasdorp - nearest hotel: Олимп
hithadhoo - nearest hotel: No hotel found
fuvahmulah - nearest hotel: Chimbalanga gardens
vilyuchinsk - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
minas de marcona - nearest hotel: Painted Pony Resort
akureyri - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
toga village - nearest hotel: No hotel found
christchurch - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
solleftea - nearest hotel: No hotel found
labrador city - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
margaret river - nearest hotel: Älggårdsberget Konferens
hami - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nuuk,GL,64.9876,-52.3907,90,No hotel found
1,jamestown,RU,66.7768,151.4367,80,No hotel found
2,puerto natales,SH,-16.4042,-6.3494,71,No hotel found
3,bredasdorp,KZ,50.2266,72.6584,36,Олимп
4,hithadhoo,CA,60.9053,-92.5890,84,No hotel found
...,...,...,...,...,...,...
150,sandakan,RU,57.9389,68.0477,93,База «Озеро Светлое»
151,chino,ZA,-29.0873,22.7568,29,No hotel found
152,marrakesh,US,58.6566,-133.3276,94,No hotel found
153,queenstown,RU,66.9908,172.4629,67,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', y='Lat',  
    geo=True,  
    tiles='OSM',  
    size=10,  
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],  
    color='blue',  
    frame_width=800,  
    frame_height=600,  
    title='City Hotel Data'  
).opts(
    projection=ccrs.PlateCarree(),  
    bgcolor='white'  
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)